# Wykład 5 04.11.2020

## Operacje wejścia/wyjścia

In [ ]:
-- Pseudo Code Pokazujący przetworzania 
main :: World -> (World, ())
main world0 =
    let (world1,a) = getChar world0
        (world2,b) = getChar world1
    in (world2,())

In [ ]:
putChar 'a'

In [ ]:
let retA = return 'a'
retA

In [ ]:
let echo = getChar >>= putChar
echo
-- >>= - Wykonuje IO, wynik jej przekazuje do putChar

In [ ]:
main :: IO()
main = putStr "Hello World!"


In [ ]:
-- Pierwsze kody są bardziej profesjonalne (do pozwala na "opuszczanie" >>)
main = 
    putStr "Hello" >> -- >> Pozwala na przejscie z IO() do a (IO() -> a)
    putStrLn " World"
-- Odpowiednik powyższego
main1 = do
    putStr "Hello"
    putStrLn " World"

main_ = putStrLn "Your name?" >> 
    getLine >>=
    \n -> putStrLn ("Hello, " ++ n)
-- Odpowiednik powyższego
main_1 = do
    putStrLn "Your name? "
    n <- getLine --   <-   - przypisanie w kontekscie monalogicznym (IO())
    putStrLn ("Hello, " ++ n)

In [ ]:
main = do
    a <- return "a" -- Nienaturalne tworzenie akcji IO() i zwracanie go do A
    b <- return "b"
    return ()
    return 1
    putStrLn $ a ++ " " ++ b 

main1 = do
    let a = "a"
        b = "b"
    return ()
    return 1
    putStrLn $ a ++ " " ++ b

### I/O: rekurencyjne deﬁnicje funkcji



In [ ]:
getLine' :: IO String
getLine' = do
    x <- getChar
    if x == '\n'
    then return []
    else do
        xs <- getLine'
        return (x:xs)
main = do
    line <- getLine'
    putStrLn line

In [ ]:
putStr' :: String -> IO ()
putStr' [] = return ()
putStr' (x:xs) = do putChar x
putStr' xs

In [ ]:
-- putStrLn na podstawie putStr
putStrLn' :: String -> IO ()
putStrLn' xs = do putStr' xs
                    putChar '\n'

### Praca z plikami 


In [ ]:
import System.Environment
import System.IO
import Data.Char(toUpper)

-- openFile :: FilePath -> IOMode -> IO Handle
main = do
    (inFileName:outFileName:_) <- getArgs
    inHdlr <- openFile inFileName ReadMode
    outHdlr <- openFile outFileName WriteMode
    inpStr <- hGetContents inHdlr
    hPutStr outHdlr (map toUpper inpStr)
    hClose inHdlr
    hClose outHdlr

In [ ]:
main = do
    (inFileName:outFileName:_) <- getArgs
    inpStr <- readFile inFileName
    writeFile outFileName (map toUpper inpStr)


### I/O i obsługa błędów

In [ ]:
import System.Environment
import System.IO
import System.IO.Error
import Control.Exception

main = do 
        (fileName:_) <- getArgs
        contents <- readFile fileName
        putStrLn $ "The file has " ++
            show (length (lines contents)) ++ " lines!"
    `catch` (\err -> if isDoesNotExistError err
            then putStrLn "The file doesn't exist!"
            else ioError err)
            
:t catch -- Exception e => IO a -> (e -> IO a) -> IO a
:t isDoesNotExistError -- IOError -> Bool


## Funktor [ Mappable ] : deﬁnicja i przykłady

In [ ]:
class Functor (f :: * -> *) where
    fmap :: (a -> b) -> f a -> f b
--fmap :: (a -> b) -> Maybe a -> Maybe b

instance Functor Maybe where
    fmap f (Just x) = Just (f x)
    fmap f Nothing = Nothing

instance Functor [] where
    fmap = map

instance Functor IO where
    fmap f action = do
        result <- action
        return (f result)

-- Functor laws
-- fmap id = id
-- fmap (g . f) = fmap g . fmap f

-- ghci> fmap (^3) (Just 4)
-- Just 64

-- ghci> fmap show [1..5]
-- "1","2","3","4","5"]

-- ghci> fmap reverse getLine
-- rats
-- star"

### Funktor: rozszerzenie DeriveFunctor


In [ ]:
data BinTree a = EmptyBT
            | NodeBT a (BinTree a) (BinTree a)
            deriving (Show)
            
instance Functor BinTree where
    fmap g EmptyBT = EmptyBT
    fmap g (NodeBT x lt rt) = NodeBT (g x) (fmap g lt) (fmap g rt)

In [ ]:
 let t1 = NodeBT 1 (NodeBT 4 EmptyBT EmptyBT) EmptyBT
 fmap (*2) t1

In [ ]:
{-# LANGUAGE DeriveFunctor #-}
data BinTree a = EmptyBT
        | NodeBT a (BinTree a) (BinTree a)
        deriving (Show, Functor)

### Funktor aplikatywny: deﬁnicja

In [ ]:
class Functor f => Applicative (f :: * -> *) where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
-- fmap :: (a -> b) -> f a -> f b

In [ ]:
-- Applicative functor laws
pure id <*> v = v -- Identity
pure f <*> pure x = pure (f x) -- Homomorphism
u <*> pure y = pure ($ y) <*> u -- Interchange
u <*> (v <*> w) = pure (.) <*> u <*> v <*> w -- Composition

fmap g x = pure g <*> x = g <$> x

### Funktory aplikatywne: przykłady

In [ ]:
instance Applicative Maybe where
    pure = Just
    Nothing <*> _ = Nothing
    (Just f) <*> w = fmap f w
(++) <$> Just "me and "<*>pure "Haskell"
Just "me and Haskell"

In [ ]:
instance Applicative IO where
    pure = return
    a <*> b = do
        f <- a
        x <- b
        return (f x)

In [ ]:
myAction = (++) <$> getLine <*> getLine
myAction
abra
kadabra
"abrakadabra"

In [ ]:
instance Applicative [] where
pure x = [x]
fs <*> xs =
[f x | f <- fs, x <- xs]

In [ ]:
import Control.Applicative
instance Applicative ZipList where
    pure x = ZipList (repeat x)
    ZipList fs <*> ZipList xs =
        ZipList (zipWith (\f x -> f x) fs xs)

### Monoid [ Appendable, Concatable ]: deﬁnicja i przykłady

In [ ]:
class Monoid a where
    mempty :: a
    mappend :: a -> a -> a
    mconcat :: [a] -> a
    {-# MINIMAL mempty, mappend #-}
    
{-
Monoid laws
mempty `mappend` x = x
x `mappend` mempty = x
(x `mappend` y) `mappend` z =
x `mappend` (y `mappend` z)
-}

In [ ]:
instance Monoid [a] where
mempty = []
mappend = (++)

-- import Data.Monoid
[1,2] `mappend` [3,4]

In [ ]:
instance Monoid b => Monoid (a->b) where
mempty _ = mempty
mappend f g x = f x `mappend` g x

mappend (*2) (+10) (Sum 2)

### Foldable

In [ ]:
class Foldable (t :: * -> *) where
    fold :: Monoid m => t m -> m
    foldMap :: Monoid m => (a -> m) -> t a -> m
    foldr :: (a -> b -> b) -> b -> t a -> b
    foldl :: (a -> b -> a) -> a -> t b -> a
    foldr1 :: (a -> a -> a) -> t a -> a
    foldl1 :: (a -> a -> a) -> t a -> a
    {-# MINIMAL foldMap | foldr #-}

In [ ]:
instance Foldable [] where
foldMap g = mconcat . map g



In [ ]:
{-# LANGUAGE DeriveFoldable #-}
data BinTree a = EmptyBT
            | NodeBT a (BinTree a) (BinTree a)
            deriving (Show, Foldable)

### Traversable [traversable functor]

In [ ]:
class (Functor t, Foldable t) => Traversable (t :: * -> *) where
    traverse :: Applicative f => (a -> f b) -> t a -> f (t b)
    sequenceA :: Applicative f => t (f a) -> f (t a)
    mapM :: Monad m => (a -> m b) -> t a -> m (t b)
    sequence :: Monad m => t (m a) -> m (t a)
    {-# MINIMAL traverse | sequenceA #-}

In [ ]:
{-# LANGUAGE DeriveFunctor, DeriveFoldable, DeriveTraversable #-}
data BinTree a = EmptyBT
                | NodeBT a (BinTree a) (BinTree a)
                deriving (Show, Functor, Foldable, Traversable)